In [ ]:
# Core 
import datetime
import os
import glob
from IPython.display import HTML, Image, display
import tempfile
import shutil

# Analysis 
import xarray as xr
import numpy as np
import pyproj as pp
import scipy as sp
import metpy as mp

# Plotting 
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from matplotlib import rcParams
import matplotlib.ticker as mticker
from matplotlib.animation import FuncAnimation
import matplotlib.colors as colors

# Debugging 
import pdb
%load_ext line_profiler

# Radar Tools
import pyart
from tint.data_utils import get_nexrad_keys, read_nexrad_key
from tint import Cell_tracks, animate
from tint.visualization import embed_mp4_as_gif

In [ ]:
lon_min = 130.40
lon_max = 131.69

lat_max = -11.61
lat_min = -12.87

CPOL_ex = xr.open_dataset(
    '/g/data/rr5/CPOL_radar/CPOL_level_1b/GRIDDED/GRID_70km_1000m/2016/20161220/CPOL_20161220_0240_GRIDS_1000m.nc'
)

In [ ]:
# Now we can easily instantiate generators of these grids like so
filenames = sorted(glob.glob('/g/data/rr5/CPOL_radar/CPOL_level_1b/GRIDDED/GRID_70km_1000m/2016/20161220/*_0*.nc'))
grids = (pyart.io.read_grid(fn, include_fields = 'reflectivity') for fn in filenames)

In [ ]:
# First, let's instantiate a tracks object and view the default parameters
tracks_obj = Cell_tracks()
tracks_obj.params

In [ ]:
# The cells we're interested in look a bit small. Let's reduce the minimum size threshold.
tracks_obj.params['MIN_SIZE'] = 16
tracks_obj.params['FIELD_THRESH'] = 40
tracks_obj.params['GS_ALT'] = 4000
# We'll give the generator of grids we made earlier to the get_tracks method of our tracks object.
tracks_obj.get_tracks(grids)

In [ ]:
# Now we can view the 'tracks' attribute of our tracks object to see the results.
tracks_obj.tracks

In [ ]:
# We can visualize these tracks to assess the performance of the algorithm.
# First we need to make another generator of the same grids for the animator
grids = (pyart.io.read_grid(fn, include_fields = 'reflectivity') for fn in filenames)

In [ ]:
# Let's save the animation in our temp directory so that it will be cleaned up later.
shutil.rmtree(tmp_dir)
tmp_dir = tempfile.mkdtemp()

animate(tracks_obj, grids, os.path.join(tmp_dir, 'tint_CPOL_anim'),
        lat_lines=np.arange(lat_min, lat_max, .1),
        lon_lines=np.arange(lon_min, lon_max, .1),
        tracers=True)

In [ ]:
# The animator saved an mp4 to our local directory. Now we can embed it as a gif in this notebook.
embed_mp4_as_gif(os.path.join(tmp_dir, 'tint_CPOL_anim.mp4'))

In [ ]:
# Let's find the cells that were tracked for the most frames
tracks_obj.tracks.groupby(level='uid').size().sort_values(ascending=False)[:10]

In [ ]:
# Now we can view these cells from a lagrangian perspective
shutil.rmtree(tmp_dir)
tmp_dir = tempfile.mkdtemp()

grids = (pyart.io.read_grid(fn) for fn in filenames)  # refresh grid generator
animate(tracks_obj, grids, os.path.join(tmp_dir, 'tint_demo_lagrangian'), style='lagrangian', uid='49', alt=2000)

In [ ]:
embed_mp4_as_gif(os.path.join(tmp_dir, 'tint_demo_lagrangian.mp4'))

In [ ]:
# We can view the attributes of this cell throughout its lifetime
tracks_obj.tracks.xs('82', level='uid')